In [ ]:
# Gender Trust Game (Croson & Buchan 1999) – LangGraph Implementation
# Baseline: faithful-to-paper design

from dataclasses import dataclass, asdict, field
from typing import List, Dict, Optional
import random
import json
import re
import pathlib
import torch
from collections import Counter
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_id = "Qwen/Qwen2.5-7B-Instruct"
tok = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,
    device_map="auto",
    torch_dtype=torch.float16,
    offload_folder="offload"
)

model.eval()

In [4]:
# -----------------------------
# 1. Data models
# -----------------------------

@dataclass
class ParticipantProfile:
    pid: str
    demographic_group_no: int
    location: str
    occupation: str
    gender: str 


@dataclass
class TrustState:
    match_id: str
    proposer: ParticipantProfile
    responder: ParticipantProfile
    transcript: List[Dict] = field(default_factory=list)

In [5]:
# -----------------------------
# 2. Load experiment JSON
# -----------------------------

EXPERIMENT_JSON_PATH = pathlib.Path("experiments_v3.json")
with open(EXPERIMENT_JSON_PATH, "r") as f:
    EXPERIMENTS = json.load(f)

EXP = next(e for e in EXPERIMENTS if e.get("experiment_id") == "gender_trust_1999")

In [6]:
def extract_demographic_info(demo_list):
    
    info = {}

    for block in demo_list:
        cat = block["category"]

        if cat == "gender":
            gender_dist = {}
            for item in block["category_info"]:
                gender_dist[item["subcategory"]] = item["percentage"]
            info["gender_distribution"] = gender_dist

        elif cat == "location":
            info["location"] = block["category_info"][0]["subcategory"]

        elif cat == "occupation":
            info["occupation"] = block["category_info"][0]["subcategory"]

    return info


In [7]:
# -----------------------------
# 3. Participant sampler
# -----------------------------

def build_participants(exp_json) -> List[ParticipantProfile]:
    
    participants = []
    pid_counter = 0
     
    for group in exp_json["demographic_info"]:
        
        n = group["num_of_participants"]
        demo = extract_demographic_info(group["demographic_info"])
    
        gender_dist = demo["gender_distribution"]
        male_pct = gender_dist["male"] / 100
        female_pct = gender_dist["female"] / 100
    
        genders = (
            ["male"] * round(n * male_pct)
            + ["female"] * round(n * female_pct)
        )
    
        # fix rounding drift
        while len(genders) < n:
            genders.append("male")
        while len(genders) > n:
            genders.pop()
    
        random.shuffle(genders)
    
        for g in genders:
            participants.append(
                ParticipantProfile(
                    pid=f"P{pid_counter:04d}",
                    demographic_group_no=group["demographic_group_no"],
                    location=demo.get("location", "unknown"),
                    occupation=demo.get("occupation", "unknown"),
                    gender=g,
                )
            )
            pid_counter += 1
    
    assert len(participants) == exp_json["total_participants_count"]
    return participants

In [8]:
participants = build_participants(EXP)

print("Total:", len(participants))
print("By group:", Counter(p.demographic_group_no for p in participants))
print("By gender:", Counter(p.gender for p in participants))

Total: 186
By group: Counter({2: 50, 1: 48, 3: 44, 4: 44})
By gender: Counter({'male': 136, 'female': 50})


In [9]:
# -----------------------------
# 4. Matchmaker (within-site random pairing)
# -----------------------------

def make_matches(participants: List[ParticipantProfile]) -> List[TrustState]:
    
    matches = []
    by_group: Dict[int, List[ParticipantProfile]] = {}
    
    for p in participants:
        by_group.setdefault(p.demographic_group_no, []).append(p)
        
    for gno, plist in by_group.items():
        random.shuffle(plist)
        assert len(plist) % 2 == 0
        
        for i in range(0, len(plist), 2):
            p1, p2 = plist[i], plist[i + 1]
            proposer, responder = random.sample([p1, p2], 2)
            
            match_id = f"G{gno}_M{i//2:03d}"
            matches.append(
            TrustState(
                match_id=match_id,
                proposer=proposer,
                responder=responder,
                )
            )
    
    return matches

In [10]:
def proposer_prompt(state: TrustState) -> str:
    p = state.proposer
    return f"""
You are participating in an economics experiment.

About you:
- Gender: {p.gender}
- Location: {p.location}
- Occupation: {p.occupation}

You are Player 1.

You are given 1000 units of money.
You may send any amount between 0 and 1000 to Player 2.
Any amount you send will be tripled before Player 2 receives it.
Player 2 may return some amount to you.

Please:
1) Decide how much you want to send.
2) Briefly explain your reason in one or two sentences.

Respond naturally, as a real participant would.
"""

In [11]:
def responder_prompt(state: TrustState, sent_operational: int) -> str:
    r = state.responder
    tripled = 3 * sent_operational
    total = 1000 + tripled

    return f"""
You are participating in an economics experiment.

About you:
- Gender: {r.gender}
- Location: {r.location}
- Occupation: {r.occupation}

You are Player 2.

Player 1 sent you {sent_operational} units.
This amount was tripled, so you received {tripled} units.
You now have a total of {total} units.

You may return any amount between 0 and {total} to Player 1.
You keep whatever you do not return.

Please:
1) Decide how much you want to return.
2) Briefly explain your reason in one or two sentences.

Respond naturally, as a real participant would.
"""


In [12]:
def extract_amount_with_llm(
    raw_text: str,
    role: str,
    min_value: int,
    max_value: int,
    model,
    tokenizer,
    temperature: float = 0.0,
) -> int | None:
    
    """
    Extracts a single integer decision from free text using an LLM.
    Returns None if extraction fails.
    """

    prompt = f"""
From the text below, extract the amount of money the person is willing to give
(send or return), not any other numbers mentioned.

Rules:
- The amount must be an integer between {min_value} and {max_value}.
- Ignore examples, explanations, or hypothetical numbers.
- If multiple amounts are mentioned, choose the one the person commits to giving.
- If no such amount is stated, output NONE.

Text:
\"\"\"
{raw_text}
\"\"\"

Output exactly one line:
AMOUNT: <integer>
or
AMOUNT: NONE
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=32,
            do_sample=False,   # deterministic for coding
            temperature=temperature,
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    answer = decoded[len(prompt):].strip()
    
    answer = re.sub(r"```.*?\n", "", answer, flags=re.DOTALL)
    answer = answer.replace("```", "").strip()

    m = re.search(r"AMOUNT:\s*(\d+|NONE)", answer, flags=re.IGNORECASE)
    
    if not m:
        return None

    val = m.group(1)

    if val.upper() == "NONE":
        return None

    try:
        val = int(val)
    except ValueError:
        return None

    if min_value <= val <= max_value:
        return val

    return None


In [13]:
def run_llm(prompt: str, max_new_tokens: int = 64) -> str:
    inputs = tok(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.3,
            top_p=0.9,
        )

    text = tok.decode(output[0], skip_special_tokens=True)
    return text[len(prompt):].strip()

In [14]:
def run_match(state: TrustState) -> TrustState:

    if state.transcript is None:
        state.transcript = []

    # --- Proposer ----------------------------------------------------------
    p_prompt = proposer_prompt(state)
    p_out = run_llm(p_prompt)

    sent_operational = extract_amount_with_llm(
        raw_text=p_out,
        role="proposer",
        min_value=0,
        max_value=1000,
        model=model,
        tokenizer=tok,
    )

    state.transcript.append({
        "role": "proposer",
        "prompt": p_prompt,
        "output": p_out,
        "sent_operational": sent_operational,
    })

    # If proposer amount is unusable, abort this match
    if sent_operational is None:
        return state


    
    # --- Responder -----------------------------------------------------------------
    r_prompt = responder_prompt(
        state,
        sent_operational=sent_operational  
    )
    r_out = run_llm(r_prompt)

    returned_operational = extract_amount_with_llm(
        raw_text=r_out,
        role="responder",
        min_value=0,
        max_value=1000 + 3 * sent_operational,
        model=model,
        tokenizer=tok,
    )

    state.transcript.append({
        "role": "responder",
        "prompt": r_prompt,
        "output": r_out,
        "returned_operational": returned_operational,
    })

    return state


In [15]:
random.seed(1)
torch.manual_seed(1)

participants = build_participants(EXP)
matches = make_matches(participants)

results = []

for m in matches: # 186 participants ⇒ 93 matches (2 per match).
    m = run_match(m)
    results.append(asdict(m))
    print(
        f"{m.match_id}: "
        f"{m.proposer.gender}@{m.proposer.location} → "
        f"{m.responder.gender}@{m.responder.location}"
    )

with open("gender_trust.json", "w") as f:
    json.dump(results, f, indent=2)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


G1_M000: male@Nankai University, China → female@Nankai University, China
G1_M001: male@Nankai University, China → female@Nankai University, China
G1_M002: female@Nankai University, China → male@Nankai University, China
G1_M003: female@Nankai University, China → male@Nankai University, China
G1_M004: female@Nankai University, China → male@Nankai University, China
G1_M005: male@Nankai University, China → male@Nankai University, China
G1_M006: male@Nankai University, China → female@Nankai University, China
G1_M007: male@Nankai University, China → male@Nankai University, China
G1_M008: female@Nankai University, China → male@Nankai University, China
G1_M009: male@Nankai University, China → male@Nankai University, China
G1_M010: male@Nankai University, China → male@Nankai University, China
G1_M011: male@Nankai University, China → male@Nankai University, China
G1_M012: male@Nankai University, China → male@Nankai University, China
G1_M013: male@Nankai University, China → male@Nankai University